In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0,1" 
import tensorflow as tf
gpu_devices = tf.config.list_physical_devices('GPU')
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)
tf.random.set_seed(22)

# Keras layers

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, RepeatVector, TimeDistributed, Bidirectional, Lambda, Masking, Multiply
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.losses import mean_squared_error
from tensorflow.keras.callbacks import ModelCheckpoint
import keras.backend as K
import pandas as pd
import numpy as np
import pickle
import multiprocessing as mp
import glob
import matplotlib.pyplot as plt

2023-08-25 20:04:44.966861: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-08-25 20:04:44.966880: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (jazz): /proc/driver/nvidia/version does not exist


In [3]:
path = '/home/chengli/dataset/df_varsAll_cleaned_withHBA1c_withBMI_pred_imp_v6.csv'
df_EHRs = pd.read_csv(path, index_col=0)

file = '~/send_chengli/kernel_matrix_allVars_cleaned_v7.pkl'
K_matrix = pd.read_pickle(file)
idps_list = K_matrix.index

#variables = ['HBA1C']
variables = ['TG', 'CREAT', 'CAC', 'COLHDL', 'COLTOT', 'COLLDL', 'HBA1C', 
             'EK201', 'EK202', 'TT103']
variables_pred = [v+'_pred_death_missings' for v in variables]
variables_pred2 = [v+'_pred' for v in variables]

generals = ['age','months_from_diag','sex','age_diag', 'ttd']
diags = ['HTA','CI','NEFRPDM','RTP_DM','NEUROPT']

col = variables_pred2


def replace_na(x):
    n_na = x.isna().sum()
    if n_na<len(variables_pred) + 2:
        for i,v in enumerate(variables_pred):
            x[v] = x[variables_pred[i]]
    return x

df_EHRs.loc[df_EHRs.ttd>0,'ttd'] = 0
df_EHRs['ttd'] = df_EHRs['ttd'].fillna(-6)
df_EHRs['ttd'] = df_EHRs['ttd']*-1
df_EHRs = df_EHRs.drop(columns = ['age_death'])
df_EHRs = df_EHRs.apply(replace_na, axis = 1)

In [8]:
np.savetxt('/home/enrico/Desktop/dm2_code/autoencoder/kernel_matrix_allVars_cleaned_v7.csv', K_matrix, delimiter=',')

In [4]:
norm_cols = variables_pred2 + ['age','months_from_diag','age_diag', 'ttd']
df_EHRs.sex = df_EHRs.sex.replace(to_replace = ['H', 'D'], value=[1,0])
df_EHRs[norm_cols] = (df_EHRs[norm_cols]-df_EHRs[norm_cols].min())/(df_EHRs[norm_cols].max()-df_EHRs[norm_cols].min())
cols_tmp = [col for col in df_EHRs.columns if col not in ['idp', 'months_from_diag']]

In [5]:
# all continues variables
col_pre = col + ['months_from_diag']
def get_real_df(idp):
    df = df_EHRs.loc[df_EHRs.idp==idp, col_pre]
    df = df.sort_values(by='months_from_diag')
    df = df.loc[:, col]
    df.loc[df_EHRs.isna().any(axis=1)] = -0.1
    #df.drop('months_from_diag', axis=1)
    return np.array(df, dtype=np.float32)
n_cpu = 10
with mp.Pool(n_cpu) as pool:
    X_set = np.stack(list((pool.map(get_real_df, idps_list))))

In [5]:
def prepare_X_set(idp):
    df = df_anal.loc[df_anal.idp==idp,cols]
    df = df.sort_values(by='months_from_diag')
    df.loc[df_anal.isna().any(axis=1)] = -0.1
    return np.array(df)

n_cpu = 10
with mp.Pool(n_cpu) as pool:
    X_set = np.stack(list((pool.map(prepare_X_set,idps_list))))

In [6]:
np.shape(X_set)

(11028, 10, 10)

In [7]:
def prepare_Y_set(idp):
    df = df_EHRs.loc[df_EHRs.idp==idp,col]
    return np.array(df)

Y_set = [X_set,K_matrix]
#with mp.Pool(n_cpu) as pool:
 #   Y_set = np.stack(list((pool.map(prepare_Y_set,idps_list))))

In [8]:
def matrices_loss(k_pred,k_true):
    
    k_pred = k_pred/tf.norm(k_pred)
    k_true = k_true/tf.norm(k_true)
    L = tf.norm(k_pred-k_true)
    return L

def vectors_loss(mask_value):
    mask_value = K.variable(mask_value)
    def masked_mse(y_true, y_pred):
        # find out which timesteps in `y_true` do not contain mascked value
        mask = K.not_equal(y_true, mask_value)
        mask = K.cast(mask, K.floatx())

        # multiply categorical_crossentropy with the mask
        loss = (y_true-y_pred)*mask
        loss = K.square(loss) 

        # take average w.r.t. the number of unmasked entries
        return K.sum(loss) / K.sum(mask)
    return masked_mse



def create_autoencoder(enc_dim = 200,repr_dim = 200, dec_dim = 200, alpha_kernel=0.5, alpha_out=1, mask_value=-0.1):
    
    # define parameters
    time_steps = pd.date_range(start='2013-01-31', end='2017-07-31', freq='6M')
    n_time_steps = len(time_steps)
    n_input = len(col)
    n_output = len(col)
    n_idps = len(idps_list)
    
    # encoder
    #inputs = Input(batch_shape=(n_idps,n_time_steps,n_input))
    inputs = Input(shape=(n_time_steps,n_input))
    #mask = Masking(mask_value=-0.1)(inputs)
    encoder_forward = LSTM(enc_dim, return_sequences=True)(inputs)
    encoder_forward = LSTM(enc_dim)(encoder_forward)
    encoder_back = LSTM(enc_dim, return_sequences=True, go_backwards=True)(inputs)
    encoder_back = LSTM(enc_dim, go_backwards=True)(encoder_back)
    encoder = Concatenate()([encoder_forward, encoder_back])
    encoder = Dense(repr_dim)(encoder)
    # kernel alligment
    kernel_alligment = Lambda(lambda x: K.dot(x,K.transpose(x)), name='kernel_output')(encoder)
    # decoder
    decoder = RepeatVector(n_time_steps)(encoder)
    decoder = LSTM(dec_dim, return_sequences=True)(decoder)
    decoder = LSTM(dec_dim, return_sequences=True)(decoder)
    decoder = TimeDistributed(Dense(n_output), name='decoder_output')(decoder)
    
    model = Model(inputs=inputs, outputs=[decoder,kernel_alligment])
    masked_mse = vectors_loss(-0.1)
    losses = {'kernel_output': matrices_loss, 'decoder_output': masked_mse}
    lossesWeight = {'kernel_output': alpha_kernel, 'decoder_output':alpha_out}

    model.compile(optimizer='adam', loss=losses, loss_weights= lossesWeight)

    
    return model

In [26]:
# define parameters
alphas = [0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]
alpha2 = ['00','01','02','03','04','05','06','07','08','09','10']  

enc_dims = [100, 150, 200]
dec_dims= [100, 150, 200]
repr_dims = [100, 150, 200]

In [ ]:
# get model
embs_done = glob.glob('/home/chengli/model/Autoencoder/embedded3/*.csv')
hist = []
for i,alpha in enumerate(alphas):
    for enc_dim in enc_dims:
        for dec_dim in dec_dims:
            for repr_dim in repr_dims:
                #chek if emb was already calculated
                embs_name = '/home/chengli/model/Autoencoder/embedded3/embs_autoencoder_allVars_clean_kernelImp_alpha{}_enc{}_dec{}_repr{}.csv'.format(alpha2[i],enc_dim,dec_dim,repr_dim)
                print('Working on variables: {}'.format(embs_name[100:-4]))
                if embs_name not in embs_done:
                    filename = '/home/chengli/model/Autoencoder/autoencoder_alpha{}_enc{}_dec{}_repr{}.h5'.format(alpha2[i],enc_dim,dec_dim,repr_dim)
                    checkpoint = ModelCheckpoint(filename, monitor='loss', verbose=0, save_best_only=True, mode='min')
                    # create model
                    model = create_autoencoder(alpha_kernel=alpha, repr_dim=repr_dim, enc_dim=enc_dim, dec_dim=dec_dim)
                    #model.summary()
                    history = model.fit(X_set, Y_set, batch_size=len(X_set), epochs=25, verbose=0, callbacks=[checkpoint])
                    train_loss = history.history['loss']
                    hist.append(train_loss[-1])
                    # get representation
                    model_autoencoder = Model(inputs= model.inputs, outputs= model.layers[6].output)
                    #model_autoencoder.summary()
                    Y_pred = model_autoencoder.predict(X_set)
                    representations = pd.DataFrame(Y_pred, index=idps_list)
                    # save embs
                    representations.to_csv(embs_name)
                    K.clear_session() 

In [31]:
model = create_autoencoder(enc_dim=200, dec_dim=150, repr_dim=200, alpha_kernel=0)

In [40]:
#model = create_autoencoder(alpha_kernel=0.5, repr_dim=100, enc_dim=200, dec_dim=200)
com_hist = model.fit(X_set, Y_set,  batch_size=len(X_set), epochs=50, verbose=1)



Epoch 1/5
1/1 [==============================] - 5s 5s/step - loss: 0.0042 - decoder_output_loss: 0.0042 - kernel_output_loss: 0.5536
Epoch 2/5
1/1 [==============================] - 5s 5s/step - loss: 0.0043 - decoder_output_loss: 0.0043 - kernel_output_loss: 0.5530
Epoch 3/5
1/1 [==============================] - 5s 5s/step - loss: 0.0042 - decoder_output_loss: 0.0042 - kernel_output_loss: 0.5526
Epoch 4/5
1/1 [==============================] - 5s 5s/step - loss: 0.0040 - decoder_output_loss: 0.0040 - kernel_output_loss: 0.5520
Epoch 5/5
1/1 [==============================] - 5s 5s/step - loss: 0.0040 - decoder_output_loss: 0.0040 - kernel_output_loss: 0.5516


In [41]:
model.save('/home/chengli/model/Autoencoder/Autoencoder.h5')